In [8]:
import numpy as np
import os
import joblib
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import ElasticNet
from sklearn.metrics import r2_score
import lightgbm as lgb

from hyperopt import hp, fmin, tpe, Trials, STATUS_OK 
# hyperopt is a better gridsearch

import warnings
warnings.filterwarnings('ignore')

###############################################################################
# STAGE 1: DEFINITIONS AND HYPERPARAMETER SPACE
###############################################################################
print("=== STAGE 1: DEFINITIONS AND HYPERPARAMETER SPACE ===")

levels_list = [5, 10, 15]
window_list = [10, 20, 50, 100, 200]
lag_list = [50, 55, 60, 65, 70, 75, 80, 85, 87]  # We'll train on these lags
MAIN_LAG = 87  # We evaluate final performance wrt y_87

# LightGBM hyperparameter search space
param_space = {
    'boosting_type':  hp.choice('boosting_type', ['dart']),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 0.6),
    'drop_rate':      hp.uniform('drop_rate', 0.4, 0.9),
    'learning_rate':  hp.loguniform('learning_rate', np.log(0.001), np.log(0.1)),
    'max_depth':      hp.choice('max_depth', [3, 4, 5, 6]),
    'min_child_samples': hp.randint('min_child_samples', 500, 30001),
    'min_child_weight':  hp.uniform('min_child_weight', 0.01, 0.05),
    'n_estimators':   hp.choice('n_estimators', [500, 1000, 1500]),
    'objective':      hp.choice('objective', ['regression']),
    'skip_drop':      hp.uniform('skip_drop', 0.5, 0.9),
    'subsample':      hp.uniform('subsample', 0.5, 1.0),
}

def train_lgb_model(params, X_train, y_train, X_val, y_val, seed=42):
    """Train LightGBM and return (model, R^2) on validation."""
    model = lgb.LGBMRegressor(
        boosting_type=params['boosting_type'],
        colsample_bytree=params['colsample_bytree'],
        drop_rate=params['drop_rate'],
        learning_rate=params['learning_rate'],
        max_depth=params['max_depth'],
        min_child_samples=params['min_child_samples'],
        min_child_weight=params['min_child_weight'],
        n_estimators=params['n_estimators'],
        objective=params['objective'],
        skip_drop=params['skip_drop'],
        subsample=params['subsample'],
        seed=seed,
        verbosity=-1,
        early_stopping_rounds=50, 
        n_jobs=-1,
    )
    model.fit(
        X_train, y_train,
        eval_set=[(X_val, y_val)],
        eval_metric='r2'
    )
    y_val_pred = model.predict(X_val)
    r2_val = r2_score(y_val, y_val_pred)
    return model, r2_val

def objective_func(space, X_train, y_train, X_val, y_val, seed=42):
    """Hyperopt objective. We want to maximize R^2 => minimize (1 - R^2)."""
    _, r2_val = train_lgb_model(space, X_train, y_train, X_val, y_val, seed)
    return {'loss': 1.0 - r2_val, 'status': STATUS_OK}

def tune_and_train(X, y, seed=42):
    """
    Splits X,y into train/val, runs Hyperopt, returns final model & R^2.
    """
    X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    trials = Trials()
    best = fmin(
        fn=lambda sp: objective_func(sp, X_tr, y_tr, X_val, y_val, seed),
        space=param_space,
        algo=tpe.suggest,
        max_evals=20, 
        trials=trials,
        rstate=np.random.default_rng(seed)
    )
    # Convert discrete choices
    best['boosting_type'] = 'dart'
    best['objective'] = 'regression'
    max_depth_values = [3, 4, 5, 6]
    n_estimators_values = [500, 1000, 1500]
    best['max_depth'] = max_depth_values[best['max_depth']]
    best['n_estimators'] = n_estimators_values[best['n_estimators']]

    final_model, final_r2 = train_lgb_model(best, X_tr, y_tr, X_val, y_val, seed)
    return final_model, final_r2, best


###############################################################################
# STAGE 2: SPLIT DATASET INTO FIRST HALF AND SECOND HALF
###############################################################################
print("\n=== STAGE 2: SPLIT DATASET INTO FIRST AND SECOND HALF ===")

# We'll get N from one sample file
sample_file = f'./data/signature_features_depth_2_levels_{levels_list[0]}_window_{window_list[0]}.npz'
with np.load(sample_file) as data:
    N = data['arr_0'].shape[0]

half_idx = N // 2

print(f"Total dataset size: {N}")
print(f"First half: [0, {half_idx}), Second half: [{half_idx}, {N})")


###############################################################################
# STAGE 3: TRAIN ONE MODEL PER (levels, window, lag) ON THE FIRST HALF
###############################################################################
print("\n=== STAGE 3: TRAINING MODELS ON THE FIRST HALF ===")

os.makedirs('./models', exist_ok=True)
trained_models = {}

for levels in levels_list:
    for window in window_list:
        # Load features for this subset
        feat_path = f'./data/signature_features_depth_2_levels_{levels}_window_{window}.npz'
        print(f"Loading features for (levels={levels}, window={window}) from {feat_path}")
        with np.load(feat_path) as fdata:
            X_sub = fdata['arr_0']  # shape: (N, D_sub)
        
        # Split & scale
        X_sub_first_half = X_sub[:half_idx]
        scaler = StandardScaler()
        X_sub_first_half_scaled = scaler.fit_transform(X_sub_first_half)
        
        for lag in lag_list:
            label_path = f'./data/targets_lag_{lag}.npz'
            with np.load(label_path) as ldata:
                y_lag = ldata['targets']  # shape: (N,)
            
            y_lag_first_half = y_lag[:half_idx]
            
            # Train + Tune
            model_lag, r2_val, best_params = tune_and_train(
                X_sub_first_half_scaled, 
                y_lag_first_half,
                seed=42
            )
            
            trained_models[(levels, window, lag)] = {
                'model': model_lag,
                'val_r2': r2_val,
                'params': best_params,
                'scaler': scaler
            }
            
            # Optionally save to disk
            model_filename = f'./models/lgb_levels_{levels}_window_{window}_lag_{lag}.joblib'
            joblib.dump(model_lag, model_filename)
            print(f"  >> TRAINED (levels={levels}, window={window}, lag={lag}) -> "
                  f"Val R^2 = {r2_val:.4f}, saved as {model_filename}")


###############################################################################
# STAGE 4: DEFINE BLENDING AND TEST PORTIONS IN THE SECOND HALF (WITH A SKIP IN-BETWEEN)
###############################################################################
print("\n=== STAGE 4: DEFINE BLENDING AND TEST PORTIONS IN SECOND HALF ===")

second_half_len = N - half_idx
blend_size = second_half_len // 2  # e.g. half for blending, half for test
blend_start = half_idx
blend_end   = blend_start + blend_size

skip_after_blend = 500  # SKIP AFTER THE BLENDING
test_start = blend_end + skip_after_blend
test_end   = N

if test_start >= test_end:
    raise ValueError("skip_after_blend is too large; no data left for testing.")

print(f"Second half length = {second_half_len}")
print(f"Blending portion: [{blend_start}, {blend_end}) => size {blend_end - blend_start}")
print(f"Skipping {skip_after_blend} after blending => test portion: [{test_start}, {test_end}) => size {test_end - test_start}")


###############################################################################
# STAGE 5: LOAD y_87 (MAIN TARGET) FOR BLENDING AND TEST
###############################################################################
print("\n=== STAGE 5: LOAD MAIN TARGET (y_87) FOR BLENDING & TEST ===")

label_87_path = f'./data/targets_lag_{MAIN_LAG}.npz'
print(f"Loading y_87 from {label_87_path}")
with np.load(label_87_path) as lbl87_data:
    y_87 = lbl87_data['targets']  # shape: (N,)

y_blend = y_87[blend_start:blend_end]
y_test  = y_87[test_start:test_end]
print(f"  >> y_blend size = {y_blend.shape[0]}, y_test size = {y_test.shape[0]}")


###############################################################################
# STAGE 6: AVERAGE ACROSS LAGS (PER SUBSET), THEN BLEND SUBSETS USING ELASTICNET
###############################################################################
print("\n=== STAGE 6: AVERAGE PREDICTIONS ACROSS LAGS, THEN BLEND SUBSETS ===")

subset_keys = list(set((lvl, w) for (lvl, w, _) in trained_models.keys()))
subset_keys.sort()

print(f"Found {len(subset_keys)} subsets: {subset_keys}")

# 6a. Blending portion: get subset predictions
blend_subset_preds = []
for (levels, window) in subset_keys:
    feat_path = f'./data/signature_features_depth_2_levels_{levels}_window_{window}.npz'
    with np.load(feat_path) as fdata:
        X_sub = fdata['arr_0']
    
    # entire second half of features
    X_sub_2nd = X_sub[half_idx:]
    
    # scale with the same scaler
    first_lag = lag_list[0]
    scaler = trained_models[(levels, window, first_lag)]['scaler']
    X_sub_2nd_scaled = scaler.transform(X_sub_2nd)
    
    # relevant indices for blending portion, relative to second half
    blend_rel_start = blend_start - half_idx
    blend_rel_end   = blend_end   - half_idx
    X_blend_portion = X_sub_2nd_scaled[blend_rel_start:blend_rel_end]
    
    # gather predictions from all lags, then average
    lag_preds_list = []
    for lag in lag_list:
        model_lag = trained_models[(levels, window, lag)]['model']
        preds_blend = model_lag.predict(X_blend_portion)
        lag_preds_list.append(preds_blend)
    
    avg_preds_blend = np.mean(lag_preds_list, axis=0)  # shape (#blend_samples,)
    blend_subset_preds.append(avg_preds_blend)

# shape (#blend_samples, #subsets)
X_blend_input = np.column_stack(blend_subset_preds)
print(f"  >> Blending input shape = {X_blend_input.shape}")

blender = ElasticNet(alpha=1.0, l1_ratio=0.5, positive=True)
blender.fit(X_blend_input, y_blend)
print("  >> [BLENDER] ElasticNet training complete.")

###############################################################################
# STAGE 7: EVALUATE THE BLENDED MODEL ON THE TEST PORTION
###############################################################################
print("\n=== STAGE 7: EVALUATE BLENDED MODEL ON THE TEST PORTION ===")

test_subset_preds = []
for (levels, window) in subset_keys:
    feat_path = f'./data/signature_features_depth_2_levels_{levels}_window_{window}.npz'
    with np.load(feat_path) as fdata:
        X_sub = fdata['arr_0']
    
    X_sub_2nd = X_sub[half_idx:]
    scaler = trained_models[(levels, window, lag_list[0])]['scaler']
    X_sub_2nd_scaled = scaler.transform(X_sub_2nd)
    
    # slice test portion in second half
    test_rel_start = test_start - half_idx
    test_rel_end   = test_end   - half_idx
    X_test_portion = X_sub_2nd_scaled[test_rel_start:test_rel_end]
    
    # average predictions across lags
    lag_preds_test_list = []
    for lag in lag_list:
        model_lag = trained_models[(levels, window, lag)]['model']
        preds_test = model_lag.predict(X_test_portion)
        lag_preds_test_list.append(preds_test)
    
    avg_preds_test = np.mean(lag_preds_test_list, axis=0)
    test_subset_preds.append(avg_preds_test)

X_test_input = np.column_stack(test_subset_preds)  # shape (#test_samples, #subsets)
y_pred_test_blend = blender.predict(X_test_input)

test_r2 = r2_score(y_test, y_pred_test_blend)
print(f"  >> [RESULT] Final Blended R^2 on Test portion (y_87) = {test_r2:.4f}")
print("\nPipeline completed successfully!")

=== STAGE 1: DEFINITIONS AND HYPERPARAMETER SPACE ===

=== STAGE 2: SPLIT DATASET INTO FIRST AND SECOND HALF ===
Total dataset size: 499000
First half: [0, 249500), Second half: [249500, 499000)

=== STAGE 3: TRAINING MODELS ON THE FIRST HALF ===
Loading features for (levels=5, window=10) from ./data/signature_features_depth_2_levels_5_window_10.npz
100%|██████████| 20/20 [21:57<00:00, 65.87s/trial, best loss: 0.980104513720027]
  >> TRAINED (levels=5, window=10, lag=50) -> Val R^2 = 0.0199, saved as ./models/lgb_levels_5_window_10_lag_50.joblib
100%|██████████| 20/20 [24:51<00:00, 74.58s/trial, best loss: 0.9825436102464474] 
  >> TRAINED (levels=5, window=10, lag=55) -> Val R^2 = 0.0175, saved as ./models/lgb_levels_5_window_10_lag_55.joblib
100%|██████████| 20/20 [25:35<00:00, 76.77s/trial, best loss: 0.9832702845751984] 
  >> TRAINED (levels=5, window=10, lag=60) -> Val R^2 = 0.0167, saved as ./models/lgb_levels_5_window_10_lag_60.joblib
100%|██████████| 20/20 [23:04<00:00, 69.20s/

KeyboardInterrupt: 